In [3]:
import numpy as np 
import pandas as pd 

In [4]:
df = pd.read_csv("./data/Instagram data.csv", encoding="latin1")
df.head()

,Impressions,From Home,From Hashtags,From Explore,From Other,Saves,Comments,Shares,Likes,Profile Visits,Follows,Caption,Hashtags
0,3920,2586,1028,619,56,98,9,5,162,35,2,Here are some of the most important data visua...,#finance #money #business #investing #investme...
1,5394,2727,1838,1174,78,194,7,14,224,48,10,Here are some of the best data science project...,#healthcare #health #covid #data #datascience ...
2,4021,2085,1188,0,533,41,11,1,131,62,12,Learn how to train a machine learning model an...,#data #datascience #dataanalysis #dataanalytic...
3,4528,2700,621,932,73,172,10,7,213,23,8,Heres how you can write a Python program to d...,#python #pythonprogramming #pythonprojects #py...
4,2518,1704,255,279,37,96,5,4,123,8,0,Plotting annotations while visualizing your da...,#datavisualization #datascience #data #dataana...


In [ ]:
#Concatenate captions and hashtags in one column
posts_text = df['Caption']+ df['Hashtags'].astype(str)
posts_text = posts_text.str.replace('\xa0', ' ', regex=True)
posts_text[0]


'Here are some of the most important data visualizations that every Financial Data Analyst/Scientist should know.#finance #money #business #investing #investment #trading #stockmarket #data #datascience #dataanalysis #dataanalytics #datascientist #machinelearning #python #pythonprogramming #pythonprojects #pythoncode #artificialintelligence #ai #dataanalyst #amankharwal #thecleverprogrammer'

In [14]:
df['instagram_post'] = posts_text

In [20]:
import pandas as pd
import gensim
import gensim.corpora as corpora
from gensim.models import CoherenceModel
import spacy
import nltk
from nltk.corpus import stopwords
import re
import matplotlib.pyplot as plt

# Download NLTK stopwords
nltk.download('stopwords')
stop_words = stopwords.words('english')

# Load data
data = df

# Preprocess the text data
def preprocess_text(text):
    text = re.sub('\s+', ' ', text)  # Remove extra spaces
    text = re.sub('\S*@\S*\s?', '', text)  # Remove emails
    text = re.sub('\'', '', text)  # Remove apostrophes
    text = re.sub('[^a-zA-Z]', ' ', text)  # Remove non-alphabet characters
    text = text.lower()  # Convert to lowercase
    return text

data['instagram_post_cleaned'] = data['instagram_post'].apply(preprocess_text)  

# Tokenize and remove stopwords
def tokenize(text):
    tokens = gensim.utils.simple_preprocess(text, deacc=True)
    tokens = [token for token in tokens if token not in stop_words]
    return tokens

data['tokens'] = data['instagram_post_cleaned'].apply(tokenize)

# Lemmatization using spaCy
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
def lemmatize(tokens):
    doc = nlp(" ".join(tokens))
    return [token.lemma_ for token in doc]

data['lemmas'] = data['tokens'].apply(lemmatize)

# Create dictionary and corpus
id2word = corpora.Dictionary(data['lemmas'])
texts = data['lemmas']
corpus = [id2word.doc2bow(text) for text in texts]

# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                            id2word=id2word,
                                            num_topics=5, 
                                            random_state=100,
                                            update_every=1,
                                            chunksize=100,
                                            passes=10,
                                            alpha='auto',
                                            per_word_topics=True)

# Print the topics
topics = lda_model.print_topics(num_words=10)
for topic in topics:
    print(topic)

# Compute coherence score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data['lemmas'], dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\yasmi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


(0, '0.084*"datum" + 0.043*"science" + 0.036*"project" + 0.028*"datascience" + 0.027*"ai" + 0.025*"amankharwal" + 0.025*"topic" + 0.024*"dataanalytic" + 0.024*"dataanalysis" + 0.024*"machinelearne"')
(1, '0.076*"python" + 0.035*"pythoncode" + 0.033*"use" + 0.032*"programming" + 0.032*"amankharwal" + 0.024*"pythonproject" + 0.021*"code" + 0.018*"language" + 0.017*"computerscience" + 0.016*"pythonlearne"')
(2, '0.112*"datum" + 0.049*"dataanalysis" + 0.049*"dataanalytic" + 0.048*"ai" + 0.048*"datascientist" + 0.048*"science" + 0.047*"datascience" + 0.045*"deeplearne" + 0.044*"amankharwal" + 0.042*"machinelearning"')
(3, '0.049*"python" + 0.042*"machine" + 0.041*"algorithm" + 0.034*"learning" + 0.033*"learn" + 0.029*"pythonproject" + 0.029*"pythoncode" + 0.026*"amankharwal" + 0.024*"use" + 0.023*"dataanalytic"')
(4, '0.054*"python" + 0.040*"pythoncode" + 0.034*"pythonproject" + 0.032*"learn" + 0.031*"ukraine" + 0.026*"use" + 0.025*"sentiment" + 0.025*"amankharwal" + 0.021*"otp" + 0.019*"ru

In [19]:
topics

[(0,
  '0.091*"datum" + 0.043*"science" + 0.036*"ai" + 0.036*"datascience" + 0.034*"amankharwal" + 0.033*"dataanalysis" + 0.033*"datascientist" + 0.033*"dataanalytic" + 0.029*"deeplearne" + 0.026*"project"'),
 (1,
  '0.075*"python" + 0.046*"pythoncode" + 0.036*"pythonproject" + 0.032*"amankharwal" + 0.030*"use" + 0.022*"programming" + 0.021*"pythonlearne" + 0.021*"pythondeveloper" + 0.019*"code" + 0.017*"language"'),
 (2,
  '0.038*"algorithm" + 0.037*"python" + 0.035*"machine" + 0.026*"learn" + 0.024*"use" + 0.024*"dataanalytic" + 0.023*"learning" + 0.023*"datum" + 0.023*"deeplearning" + 0.023*"datascientist"')]